In [ ]:
# With systematics!

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import localSettings as ls
from load_data_run123 import *
from unblinding_far_sideband import *
from matplotlib import pyplot as plt
from matplotlib import gridspec
from scipy.optimize import curve_fit
import seaborn as sns
from ccncpi0_analysis import ccncpi0_analysis
from sys_functions import *

In [ ]:
# ntuple_path = ls.ntuple_path+"unfiltered/"
ntuple_path = ls.ntuple_path+"twoshrs/"

In [ ]:
my_ccncpi0_analysis = ccncpi0_analysis(ntuple_path)

In [ ]:
# Efficiency plot
my_ccncpi0_analysis.efficiency_plot(variable='pi0_e', bin_edges=[0.1, 0.25, 0.4, 0.7, 1.5])

In [ ]:
# this is a function which gives as output [num_CC, den_CC, eff_CC, num_NC, den_NC, eff_NC, eff_NC/eff_CC]
# we are going to give it as an argument to sampleSystematics in the following cell
Eff_ccnc(my_ccncpi0_analysis.pi0s, 
                          weights=my_ccncpi0_analysis.pi0s["weightSplineTimesTune"], 
                          var='pi0_e',
                          num_query=my_ccncpi0_analysis.CCNCSEL, 
                          den_query=my_ccncpi0_analysis.ACCEPTANCE, 
                          bin_edges=[0.1, 1.5], #only one bin, overall efficiency
                          num_dem=True)

In [ ]:
cv, variations = sampleSystematics(my_ccncpi0_analysis.pi0s, 
                  Eff_ccnc, 
                  var_weight_sys="weightsGenie", #"weightsFlux"
                  var_weight_cv='weightSplineTimesTune', 
                  n_max_universes=None, 
                  var='pi0_e',
                  num_query=my_ccncpi0_analysis.CCNCSEL, 
                  den_query=my_ccncpi0_analysis.ACCEPTANCE, 
                  bin_edges=[0.1, 1.5],
                  num_dem=True)
delta_eff = (variations-cv)
# I build the dataset without subtracting the CV, it's a matter of taste in the plot
eff_df = pd.DataFrame((variations), columns=['Num CC', 'Den CC', 'Eff CC', 'Num NC', 'Den NC', 'Eff NC', 'Eff ratio'])

In [ ]:
# More info here -> https://seaborn.pydata.org/tutorial/axis_grids.html
g = sns.PairGrid(eff_df)
g.map_diag(sns.histplot)
g.map_offdiag(sns.scatterplot)

In [ ]:
# Covariance matrix, visualising the same function
cov = covMatrix(my_ccncpi0_analysis.pi0s, 
                  Eff_ccnc, 
                  var_weight_sys="weightsGenie", 
                  var_weight_cv='weightSplineTimesTune', 
                  n_max_universes=None, 
                  var='pi0_e',
                  num_query=my_ccncpi0_analysis.CCNCSEL, 
                  den_query=my_ccncpi0_analysis.ACCEPTANCE, 
                  bin_edges=[0.1, 1.5],
                  num_dem=True)
diag_uncertainties = np.sqrt(np.diag(cov))[:, np.newaxis]
corr = cov / (diag_uncertainties.T * diag_uncertainties)

In [ ]:
# Plot a correlation matrix
plt.figure(figsize=(14, 14))
hm = sns.heatmap(corr,
                 cbar=True,
                 annot=True,
                 square=True,
                 fmt='.3f',
                 annot_kws={'size': 12},
                 xticklabels=['Num CC', 'Den CC', 'Eff CC', 'Num NC', 'Den NC', 'Eff NC', 'Eff ratio'],
                 yticklabels=['Num CC', 'Den CC', 'Eff CC', 'Num NC', 'Den NC', 'Eff NC', 'Eff ratio'],
                 )
plt.title('CC-NC$\pi_0$ selection efficiency\ncorrelation matrix')